In [5]:
%load_ext autoreload
%autoreload 2

/home/peshkichev/miniconda3/envs/py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(
[autoreload of dialogue2graph.metrics.llm_metrics.metrics failed: Traceback (most recent call last):
  File "/home/peshkichev/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/peshkichev/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/peshkichev/miniconda3/envs/py311/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from dialogue2graph.datasets.complex_dialogues.generation import LoopedGraphGenerator

In [18]:
from dialogue2graph.pipelines.core.graph import Graph
from dialogue2graph.pipelines.core.dialogue_sampling import RecursiveDialogueSampler
from dialogue2graph.datasets.complex_dialogues.prompts import graph_example

In [19]:
sampler = RecursiveDialogueSampler()

In [20]:
dias = sampler.invoke(Graph(graph_example),5)

ENDS:  [9]
NODES:  [[1, 2, 5, 10, 11, 6, 7, 9], [1, 5, 10, 11, 6, 7, 9], [1, 2, 5, 7, 9], [1, 3, 8, 9], [1, 4, 8, 9], [1, 5, 7, 9]]
REM:  [[1, 5, 10, 11, 6, 7, 9], [1, 2, 5, 7, 9], [1, 3, 8, 9], [1, 4, 8, 9]]
Graph has all the dialogues


In [9]:
type(dias[0])

dialogue2graph.pipelines.core.dialogue.Dialogue

In [10]:
from typing import Optional
from pydantic_settings import BaseSettings, SettingsConfigDict


class EnvSettings(BaseSettings, case_sensitive=True):

    model_config = SettingsConfigDict(env_file="./.env", env_file_encoding="utf-8")

    OPENAI_API_KEY: Optional[str]
    OPENAI_BASE_URL: Optional[str]
    HUGGINGFACE_TOKEN: Optional[str]
    EMBEDDER_DEVICE: Optional[str]

In [11]:
env_settings = EnvSettings()

In [12]:
from langchain.chat_models import ChatOpenAI

In [13]:
generation_model = ChatOpenAI(model="chatgpt-4o-latest", api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL, temperature=1)
validation_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL)

/tmp/ipykernel_3695116/2894222870.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  generation_model = ChatOpenAI(model="chatgpt-4o-latest", api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL, temperature=1)


In [25]:
generator = LoopedGraphGenerator(generation_model,validation_model)

In [26]:
graph = generator.invoke("booking a table")

INFO:dialogue2graph.datasets.complex_dialogues.generation:Generating Graph ...
INFO:dialogue2graph.utils.prompt_caching:Generated UUID: 00000000-0000-0000-0000-00000000002a



Generating graph for topic: booking a table


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dialogue2graph.datasets.complex_dialogues.generation:Graph generated is {'edges': [{'source': 1, 'target': 2, 'utterances': ["I'd like to book a table.", 'Can you help me find a restaurant and reserve a table?']}, {'source': 2, 'target': 3, 'utterances': ["I'm looking for an Italian restaurant downtown.", 'I want to book a table in a sushi place near the river.']}, {'source': 3, 'target': 4, 'utterances': ['I want it for Saturday at 7 PM.', 'Can I book it for tomorrow evening around 8?']}, {'source': 4, 'target': 5, 'utterances': ["It'll be for 4 people.", 'Just 2 of us.']}, {'source': 5, 'target': 9, 'utterances': ["No, that's everything. Thanks!", 'All good, I appreciate it!']}, {'source': 5, 'target': 8, 'utterances': ["Can you give me the restaurant's address and phone number?", 'Could you share the contact details of the place?']}, {'source': 5, 'target': 6, 'ut

ENDS:  [9]
NODES:  [[1, 2, 3, 4, 5, 6, 5, 7, 5, 8, 9], [1, 2, 3, 4, 5, 7, 5, 6, 5, 8, 9], [1, 2, 3, 4, 5, 6, 5, 7, 5, 9], [1, 2, 3, 4, 5, 7, 5, 6, 5, 9], [1, 2, 3, 4, 5, 6, 5, 8, 9], [1, 2, 3, 4, 5, 7, 5, 8, 9], [1, 2, 3, 4, 5, 6, 5, 9], [1, 2, 3, 4, 5, 7, 5, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 3, 4, 5, 9], [1, 2, 9]]
REM:  [[1, 2, 3, 4, 5, 6, 5, 9], [1, 2, 3, 4, 5, 7, 5, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 9]]


INFO:dialogue2graph.datasets.complex_dialogues.generation:Sampled 65 dialogues


Graph has all the dialogues
Graph has all the dialogues


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dialogue2graph.datasets.complex_dialogues.generation:Validating and fixing transitions...
INFO:dialogue2graph.datasets.complex_dialogues.generation:Validating initial graph
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx

Sampling dialogues...
ENDS:  [9]
NODES:  [[1, 2, 3, 4, 5, 6, 5, 7, 5, 8, 9], [1, 2, 3, 4, 5, 7, 5, 6, 5, 8, 9], [1, 2, 3, 4, 5, 6, 5, 7, 5, 9], [1, 2, 3, 4, 5, 7, 5, 6, 5, 9], [1, 2, 3, 4, 5, 6, 5, 8, 9], [1, 2, 3, 4, 5, 7, 5, 8, 9], [1, 2, 3, 4, 5, 6, 5, 9], [1, 2, 3, 4, 5, 7, 5, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 3, 4, 5, 9], [1, 2, 9]]
REM:  [[1, 2, 3, 4, 5, 6, 5, 9], [1, 2, 3, 4, 5, 7, 5, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 9]]


INFO:dialogue2graph.datasets.complex_dialogues.generation:going to return: {'edges': [{'source': 1, 'target': 2, 'utterances': ["I'd like to book a table.", 'Can you help me find a restaurant and reserve a table?']}, {'source': 2, 'target': 3, 'utterances': ["I'm looking for an Italian restaurant downtown.", 'I want to book a table in a sushi place near the river.']}, {'source': 3, 'target': 4, 'utterances': ['I want it for Saturday at 7 PM.', 'Can I book it for tomorrow evening around 8?']}, {'source': 4, 'target': 5, 'utterances': ["It'll be for 4 people.", 'Just 2 of us.']}, {'source': 5, 'target': 8, 'utterances': ["Can you give me the restaurant's address and phone number?", 'Could you share the contact details of the place?']}, {'source': 5, 'target': 6, 'utterances': ['Actually, can I change the time to 8 PM?', 'Wait, can you book it for next Friday instead?']}, {'source': 6, 'target': 5, 'utterances': ['Make it 9 PM, please.', "Let's move it to Sunday at noon."]}, {'source': 5,

Graph has all the dialogues
Sampled 65 dialogues
Graph has all the dialogues


In [16]:
graph

/home/peshkichev/miniconda3/envs/py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "DialogueBase"
  warnings.warn(


[]